In [1]:
import pickle
import numpy as np

In order to slowly move towards a modular file structure, we define constants, the model parameters, and ohters in separate notebooks. This notebook uses the ipynb library (pip install ipynb) to import constants, model parameters, and funtions that have been moved to other notebooks. The goal of this restructuring is to improve readability, make dependences clear and ease settig up testing tear.

In [2]:
# Import specified definitions only from given notebook
import ipynb.fs
from .defs.shared_constants import MISSING_INT
from .defs.read import read_init_file
from .defs.read import init_dict_to_attr_dict

In [3]:
# Read in initialization file as attr_dict
attr_dict = read_init_file('toy_model_init_file.yml')

In this function I need to unpack num_periods, num_choices, educ_max, educ_min, educ_range. What is a more efficient way to do this, e.g. using kwargs?

In [4]:
def pyth_create_state_space(attr_dict):
    """Create grid for state space."""

    # Unpack parameter from the model specification
    num_choices = attr_dict['GENERAL']['num_choices']
    num_periods = attr_dict['GENERAL']['num_periods']
    educ_max = attr_dict['INITIAL_CONDITIONS']['educ_max']
    educ_min = attr_dict['INITIAL_CONDITIONS']['educ_min']
    educ_range = attr_dict['DERIVED_ATTR']['educ_range']

    # Array for mapping the state space points to indices
    shape = (num_periods, educ_range, num_choices, num_periods, num_periods)
    mapping_state_index = np.tile(MISSING_INT, shape)

    # Maximum number of state space points per period. There can be no more states in a period than this number. 
    num_states_period_upper_bound = np.prod(mapping_state_index.shape)

    # Array to collect all state space points (states) that can be reached each period
    states_all = np.tile(MISSING_INT, (num_periods, num_states_period_upper_bound, 4))

    # Array for the maximum number state space points / states per period
    states_number_period = np.tile(MISSING_INT, num_periods)


    # Loop over all periods / all ages
    for period in range(num_periods):

        # Start count for admissible state space points
        k = 0

        # Loop over all possible initial conditions for education
        for educ_years in range(educ_range):

            # Check if individual has already completed education
            # and will make a labor supply choice in the period
            if educ_years > period:
                continue

            # Loop over all admissible years of experience accumulated in part-time
            for exp_f in range(num_periods):

                # Loop over all admaissible years of experience accumulated in full-time
                for exp_p in range(num_periods):

                    # The accumulation of experience cannot exceed time lapsed
                    # since individual entered the model
                    if (exp_f + exp_p > period - educ_years):
                        continue

                    # Add an additional entry state [educ_years + educ_min, 0, 0, 0]
                    # for individuals who have just completed education
                    # and still have no experience in any occupation.
                    if (period == educ_years):

                        # Assign an additional the integer count k
                        # for entry state
                        mapping_state_index[
                            period,
                            educ_years,
                            0,
                            0,
                            0,
                        ] = k

                        # Record the values of the state space components
                        # for the currentry reached entry state
                        states_all[period, k, :] = [
                            educ_years + educ_min,
                            0,
                            0,
                            0,
                        ]

                        # Update count once more
                        k += 1

                    else:

                        # Loop over the three labor market choices, N, P, F
                        for choice_lagged in range(num_choices):

                            # If individual has only worked full-time in the past,
                            # she can only have full-time (2) as lagged choice
                            if (choice_lagged != 2) and (exp_f == period - educ_years):
                                continue

                            # If individual has only worked part-time in the past,
                            # she can only have part-time (1) as lagged choice
                            if (choice_lagged != 1) and (exp_p == period - educ_years):
                                continue

                            # If an individual has never worked full-time,
                            # she cannot have that lagged activity
                            if (choice_lagged == 2) and (exp_f == 0):
                                continue

                            # If an individual has never worked part-time,
                            # she cannot have that lagged activity
                            if (choice_lagged == 1) and (exp_p == 0):
                                continue

                            # Check for duplicate states
                            if (
                                mapping_state_index[
                                    period,
                                    educ_years,
                                    choice_lagged,
                                    exp_p,
                                    exp_f,
                                ]
                                != MISSING_INT
                            ):
                                continue

                            # Assign the integer count k as an indicator for the
                            # currently reached admissible state space point
                            mapping_state_index[
                                period,
                                educ_years,
                                choice_lagged,
                                exp_p,
                                exp_f,
                            ] = k

                            # Record the values of the state space components
                            # for the currently reached admissible state space point
                            states_all[period, k, :] = [
                                educ_years + educ_min,
                                choice_lagged,
                                exp_p,
                                exp_f,
                            ]

                            # Update count
                            k += 1

        # Record number of admissible state space points for the period currently reached in the loop 
        states_number_period[period] = k

    # Auxiliary objects
    max_states_period = max(states_number_period)

    # Collect arguments
    args = (states_all, states_number_period, mapping_state_index, max_states_period)
    
    # Record function output
    return args

In [5]:
args = pyth_create_state_space(attr_dict)

In [6]:
# Unpack objects from agrs
states_all, states_number_period, mapping_states_index, max_states_period = args[0], args[1], args[2], args[3]

In [7]:
states_number_period

array([  1,   4,  14,  34,  67, 115, 180, 260, 355, 465])

Ideas checks and tests:
- check if loop visits the whole range of values by checking min and max of array entries for the differnt state space components.

In [8]:
print(np.amax(states_all[:,:,0]), np.amax(states_all[:,:,1]), np.amax(states_all[:,:,2]), np.amax(states_all[:,:,3]))

14 2 9 9


Export final output:

In [9]:
# Choose a file name
file_name = "args_file.pkl"

# Open the file for writing
with open(file_name,'wb') as my_file_obj:
    pickle.dump(args, my_file_obj)